# Решающие деревья
В этой части анализа я полностью разберусь в алгоритме решающих деревьев и их "агрегаторе" -- методе построения случайного леса

In [91]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score # AUC для оценки качества модели
from sklearn.metrics import confusion_matrix # для определения FP, FN, TP, TN

In [92]:
# данные для обучения модели
shots = pd.read_csv('curry1415.csv', header=0)
data = shots.sort_values(['espn_game_id', 'period', 'minutes_remaining', 'seconds_remaining'], ascending=[1, 1, 0, 0])
data.index=range(len(data))


In [93]:
# В качестве данных для тестирования будет использоваться сезон 15-16. Вроде, Стефан Карри
# не изменил свой стиль игры и не получал травм, поэтому можно тестировать результаты на другом сезоне
# данные для тестирования
shots2 = pd.read_csv('curry1516.csv', header=0)
test = shots2.sort_values(['espn_game_id', 'period', 'minutes_remaining', 'seconds_remaining'], ascending=[1, 1, 0, 0])
test.index=range(len(test))


In [94]:
# Создадим более точные переменные расстояния (в данных они округленные)
x=np.array(abs(data['x']))
y=np.array(abs(data['y']))
data['distance'] = ((x**2+y**2)**0.5)/10

xx=np.array(abs(test['x']))
yy=np.array(abs(test['y']))
test['distance'] = ((xx**2+yy**2)**0.5)/10

Подумаем, какие переменные могут быть использованы в качестве регрессоров. Это точно:
* Координаты броска
* Тип броска 
* Расстояние (мультколлиниарность)

Хотя лучше подумать, что точно не подойдет, а остальное засунуть в модель:
* name
* team_name
* game_date 
* season
* espn_player_id
* team_id
* espn_game_id

Остальные переменные будем использовать, но стоит помнить, что есть пропуски по именам защитников

In [95]:
# разделим выборку на таргетируемую переменную и все остальные
train = data.drop('shot_made_flag', 1)
train_y = data['shot_made_flag']

# дропнем лишние переменные
drops = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', 'espn_game_id']
for drop in drops:
    train= train.drop(drop, 1)

In [96]:
# категориальные переменные нужно разбить на дамми, потому что этого требует алгоритм случайного леса, реализованный на sklearn
categorical_vars = ['action_type', 'defender_name', 'shot_type', 'opponent']
for var in categorical_vars:
    train = pd.concat([train, pd.get_dummies(train[var], prefix=var)], 1)
    train = train.drop(var, 1)
# посмотрим, к каким типам принадлежат переменные
# train.dtypes

In [97]:
# тоже самое для тестовой выборки
testing = test.drop('shot_made_flag', 1)
testing_y = test['shot_made_flag']

# дропнем лишние переменные
drops = ['name', 'team_name', 'game_date', 'season', 'espn_player_id', 'team_id', 'espn_game_id']
for drop in drops:
    testing= testing.drop(drop, 1)
    
for var in categorical_vars:
    testing = pd.concat([testing, pd.get_dummies(testing[var], prefix=var)], 1)
    testing = testing.drop(var, 1)

Построим случайный лес с неоптимизированными параметрами

In [98]:
model = RandomForestClassifier(n_estimators=100, max_depth=10)
model.fit(train, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Предсказать не получается, так как типов бросков в разных сезонах не совпадают. Возможно, часть множеств не пересекается, поэтому алгоритм не может классифицировать. Пока оставим эту проблему и попробуем проверить точность классификации при помощи разбиения обучающей выборки на две части. Позже: проблема решается через добавление недостающих дамми-переменных в обе выборки